In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as mt
from numpy import genfromtxt
from numpy import linalg as LA
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from numpy import ma

from sklearn.ensemble import RandomForestClassifier

import graphviz
from sklearn import tree

In [111]:
df = pd.read_csv('OnlineNewsPopularity.csv')
df = df.loc[:, ' timedelta':]
dfx = df.loc[:, ' timedelta':' abs_title_sentiment_polarity']
dfy = df.loc[:, ' shares']

data = df.as_matrix(columns=None)

data = data[data[:,3]<1]

c = np.array([10,19,43,44,45,47,48,49,52])
for i in c:
    data = data[data[:,i]!=0]

'''
c = np.array([2,6,7,8,9,21,25,26,27,28,29,38,39,40,41,42,46])
for i in c:
    if (LA.norm(data[:,i],0)==data[:,i].shape):
        data[:,i] = np.log(data[:,i])
    else:
        data[:,i] = np.sqrt(data[:,i])
'''
# eliminate unvalid data
#c = np.array([18,20,22,24])
#data = np.delete(data, c, 1)

# pre eliminate unvalid and unuseful features
#c = np.array([12,13,14,15,16,17,18,20,22,24,30,31,32,33,34,35,36,37])
#data = np.delete(data, c, 1)

my_data = np.copy(data)
datax = my_data[:,:my_data.shape[1]-1]
datay = my_data[:,my_data.shape[1]-1]
for j in range(0,datay.shape[0]):
    if datay[j] >1400:
        datay[j] =1
    else:
        datay[j] =0
        
datax_train = datax[:int(datax.shape[0]*3/4),:]
datay_train = datay[:int(datay.shape[0]*3/4)]

datax_test = datax[int(datax.shape[0]*3/4):,:]
datay_test = datay[int(datay.shape[0]*3/4):]

In [114]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(datax_train, datay_train)
clf.score(datax_test,datay_test)

0.64439298709890835

In [116]:
features = dfx.columns
targetnames = ['nonpopular','popular']
dot_data = tree.export_graphviz(clf.estimators_[0], out_file=None,feature_names=features,class_names=targetnames,  
                         filled=True, rounded=True,  
                         special_characters=True) 
graph = graphviz.Source(dot_data) 
graph.render("randomforestwithprepocess")

'randomforestwithprepocess.pdf'

In [107]:
targetnames = ['popular','nonpopular']

In [115]:
clf.estimators_[0]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=209652396, splitter='best')

In [125]:
clf.feature_importances_

array([ 0.00227703,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.01935766,  0.14502444,  0.        ,  0.0214695 ,
        0.        ,  0.01899416,  0.        ,  0.00482539,  0.02646022,
        0.09719766,  0.24833905,  0.07744565,  0.03711625,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.07218761,  0.        ,  0.02008698,  0.        ,  0.        ,
        0.11996659,  0.        ,  0.05703874,  0.        ,  0.0024535 ,
        0.        ,  0.        ,  0.02494404,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.00481552,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

In [126]:
np.sum(clf.feature_importances_)

1.0